# Install and import libraries

In [1]:
!pip install einops
!pip install datasets
!pip install torch-fidelity
!pip install torchmetrics
!pip3 install image-similarity-measures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 522 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 13.8 MB/s 
     |████████████████████████████████| 115 kB 56.4 MB/s 
     |████████████████████████████████| 182 kB 59.7 MB/s 
     |████████████████████████████████| 212 kB 64.0 MB/s 
     |████████████████████████████████| 127 kB 91.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 13.7 MB/s 
Looking in indexes: https://pypi.org/simp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import numpy
from numpy import cov, trace, iscomplexobj
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import sys

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Paths (Checkpoint)

In [2]:
DiffusionModel_checkpoint = "/content/drive/MyDrive/Colab Notebooks/DiffusionModels/CIFAR10/checkpoint_T1000/cifar10/Unet.ckpt"

Stats_path = "/content/drive/MyDrive/Colab Notebooks/DiffusionModels/CIFAR10/checkpoint_T1000/cifar10/stats_Cifar10_T1000.csv"

# Load FashionMNIST Dataset to get Real Images

In [3]:
train_set = torchvision.datasets.CIFAR10("./data", download=True, transform=
                                                transforms.Compose([transforms.ToTensor()]))
train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=100, shuffle=False)

Files already downloaded and verified


# Generate Images from trained Diffusion Model

In [6]:
!git clone https://github.com/GangaMegha/Diffusion-Models.git 

Cloning into 'Diffusion-Models'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 191 (delta 105), reused 136 (delta 56), pack-reused 0
Receiving objects: 100% (191/191), 49.31 KiB | 9.86 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [4]:
%cd /content/Diffusion-Models/src
!ls

/content/Diffusion-Models/src
config.py	diffusion  main.py  __pycache__  visualize
data_loader.py	logger.py  models   train


#### Load Model

In [5]:

from models.unet import Unet as Model
from config import LEVEL, TRAIN, MODEL, MODEL_TYPE, DATASET_NAME, CONFIG, CHECKPOINT_PATH, RESULT_PATH, LOG_PATH

from diffusion.reverse_data_generate import sample
from diffusion.var_schedule import alpha_beta

from data_loader import reverse_transform

cfg = CONFIG[MODEL_TYPE.value]["cifar10"]
Noise_model = Model(cfg)
Noise_model_checkpoint = torch.load(DiffusionModel_checkpoint)
Noise_model.load_state_dict(Noise_model_checkpoint['state_dict'])
Noise_model.to(device)
Noise_model.eval()

variance_dict = alpha_beta(cfg.get('T'), schedule=cfg.get("var_schedule"))

#### Generate Images

In [6]:
fake_images = []
for i in range(10):
  fake_images_list = sample(Noise_model, variance_dict, cfg, sample_cnt=100, every50=False)
  fake_images.append(reverse_transform()(torch.cat(fake_images_list, dim=0)))

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

# Compute True FID and IS using Inception Network

In [7]:
from train.metrics import FID, IS

In [8]:
FID_ = FID().to(device)
IS_ = IS().to(device)

Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth


  0%|          | 0.00/91.2M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [9]:
del Noise_model

In [10]:
it = iter(train_loader)
FID_list, IS_list = [], []

for i in range(10):
  batch = next(it)[0]
  FID_list.append(FID_(fake_images[i].to(device), (batch*255).to(dtype=torch.uint8).to(device), cfg["grayscale"]).item())
  IS_list.append(IS_(fake_images[i].to(device), cfg["grayscale"]).item())

print("FID_list : ", FID_list)
print("IS_list : ", IS_list)

print("FID Mean : ", np.mean(np.array(FID_list)))
print("FID Std : ", np.std(np.array(FID_list)))

print("IS Mean : ", np.mean(np.array(IS_list)))
print("IS Std : ", np.std(np.array(IS_list)))

FID_list :  [191.23182678222656, 187.6072235107422, 189.94952392578125, 200.1397705078125, 200.43246459960938, 198.7239532470703, 189.56431579589844, 193.8949432373047, 194.73109436035156, 186.0653533935547]
IS_list :  [3.522639513015747, 3.5197465419769287, 3.6538307666778564, 3.701427459716797, 3.417789936065674, 3.596240282058716, 3.5319721698760986, 3.4893815517425537, 3.6800644397735596, 3.8330419063568115]
FID Mean :  193.23404693603516
FID Std :  4.9359746117682475
IS Mean :  3.594613456726074
IS Std :  0.11666966584818994


# Save Computed stats

In [11]:
# load csv module
import csv

# define a dictionary with key value pairs
diction = {
    "Metric" : "Value",
    "FID Mean" : np.mean(np.array(FID_list)),
    "FID Std" : np.std(np.array(FID_list)),
    "IS Mean" : np.mean(np.array(IS_list)),
    "IS Std" : np.std(np.array(IS_list))
    }

# open file for writing, "w" is writing
w = csv.writer(open(Stats_path, "w"))

# loop over dictionary keys and values
for key, val in diction.items():

    # write every key and value to file
    w.writerow([key, val])


# Visualize generated samples

In [12]:
from torchvision.utils import save_image
all_images = torch.cat(fake_images_list, dim=0)
save_image((all_images[:8*4]+1)*0.5, "Cifar10_FakeImages.png", nrow=8)

In [18]:
from torchvision.utils import save_image
all_images = torch.cat(fake_images_list, dim=0)
save_image((all_images[:12*6]+1)*0.5, "Cifar10_FakeImages_big.png", nrow=12)